In [1]:
### imports
import warnings
warnings.simplefilter('ignore')
import itertools
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.neuralnet.utilities import mean_ci
from mliv.neuralnet import AGMMEarlyStop as AGMM
from mliv.neuralnet.moments import avg_small_diff
from sklearn.ensemble import RandomForestRegressor
import joblib
import pandas as pd
from collections import OrderedDict
import scipy


def plot_results(fname, n, iv_strength, dr, tmle, ipw, direct, true):
    plt.title(f'fname={fname}, n={n}, strength={iv_strength}, true={true:.3f}\n'
              f'dr: Cov={np.mean((dr[:, 1] <= true) & (true <= dr[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((dr[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((dr[:, 0]-true)):.3f}\n'
              f'tmle: Cov={np.mean((tmle[:, 1] <= true) & (true <= tmle[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((tmle[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((tmle[:, 0]-true)):.3f}\n'
              f'ipw: Cov={np.mean((ipw[:, 1] <= true) & (true <= ipw[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((ipw[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((ipw[:, 0]-true)):.3f}\n'
              f'direct: Cov={np.mean((direct[:, 1] <= true) & (true <= direct[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((direct[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((direct[:, 0]-true)):.3f}\n')
    plt.hist(dr[:, 0], label='dr')
    plt.hist(tmle[:, 0], label='tmle', alpha=.4)
    plt.hist(ipw[:, 0], label='ipw', alpha=.4)
    plt.hist(direct[:, 0], label='direct', alpha=.4)
    plt.legend()

n_z = 1
n_t = 1
dgp_num = 5
epsilon = 0.1 # average finite difference epsilon
moment_fn = lambda x, fn, device: avg_small_diff(x, fn, device, epsilon)

In [22]:
for clever in [False, True]:
    resd = {}
    for n_t in [1]:
        resd[n_t] = {}
        for fname in ['abs', '2dpoly']:
            resd[n_t][fname] = OrderedDict()
            for n in [500, 1000, 2000]:
                lambda_l2_h = .1/n**(.9)
                print(lambda_l2_h)
                nkey = f'$n={n}$'
                resd[n_t][fname][nkey] = {}
                for iv_strength in [0.2, 0.5]:
                    ivkey = f'$\\pi={iv_strength}$'
                    resd[n_t][fname][nkey][ivkey] = {}
                    for endogeneity_strength in [0.05, 0.1]:
                        true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                        ekey = f'$\\rho={endogeneity_strength}$'
                        resd[n_t][fname][nkey][ivkey][ekey] = {}
                        for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                            data = np.array([r[it] for r in results])
                            if method in ['dr', 'tmle']:
                                cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                            else:
                                cov = 'NA'
                            resd[n_t][fname][nkey][ivkey][ekey][method] = {
                                            'cov': cov,
                                            'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                            'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}',
                                            'std': f'{np.std(data[:, 0]):.3f}'}
                        resd[n_t][fname][nkey][ivkey][ekey] = pd.DataFrame(resd[n_t][fname][nkey][ivkey][ekey])
                    resd[n_t][fname][nkey][ivkey] = pd.concat(resd[n_t][fname][nkey][ivkey], sort=False)
                resd[n_t][fname][nkey] = pd.concat(resd[n_t][fname][nkey], sort=False)
            resd[n_t][fname] = pd.concat(resd[n_t][fname], sort=False)
        resd[n_t] = pd.concat(resd[n_t], sort=False)
    display(pd.concat(resd).unstack(level=5))
    print(pd.concat(resd).unstack(level=5).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='lll||lll|lll|lll|lll|',
                                                    multicolumn_format='c|'))

0.00037232911332721384
0.00019952623149688796
0.00010692345999911879
0.00037232911332721384
0.00019952623149688796
0.00010692345999911879


dr                      tmle          \
                                       cov   rmse   bias    std  cov    rmse   
1 abs    $n=500$  $\pi=0.2$ $\rho=0.3$  94  0.151  0.018  0.150   90   0.175   
                            $\rho=0.8$  90  0.147  0.010  0.146   80  46.310   
                  $\pi=0.5$ $\rho=0.3$  93  0.066  0.007  0.066   92   0.069   
                            $\rho=0.8$  93  0.064  0.009  0.063   91   0.071   
         $n=1000$ $\pi=0.2$ $\rho=0.3$  95  0.115  0.012  0.115   92   0.125   
                            $\rho=0.8$  90  0.131  0.009  0.131   84   0.202   
                  $\pi=0.5$ $\rho=0.3$  94  0.048  0.005  0.048   94   0.049   
                            $\rho=0.8$  93  0.047  0.007  0.046   92   0.049   
         $n=2000$ $\pi=0.2$ $\rho=0.3$  96  0.080  0.009  0.079   94   0.082   
                            $\rho=0.8$  94  0.096  0.010  0.095   90   0.116   
                  $\pi=0.5$ $\rho=0.3$  96  0.031  0.003  0.031   96   0.032   
                            $\rho=0.8$  96  0.030  0.003  0.030   95   0.031   
  2dpoly $n=500$  $\pi=0.2$ $\rho=0.3$  91  0.240  0.005  0.240   91   0.263   
                            $\rho=0.8$  85  0.190  0.033  0.187   80  10.316   
                  $\pi=0.5$ $\rho=0.3$  91  0.083  0.002  0.083   91   0.083   
                            $\rho=0.8$  89  0.078  0.002  0.078   89   0.081   
         $n=1000$ $\pi=0.2$ $\rho=0.3$  92  0.181  0.008  0.181   92   0.189   
                            $\rho=0.8$  89  0.150  0.012  0.149   84   0.195   
                  $\pi=0.5$ $\rho=0.3$  92  0.058  0.005  0.058   92   0.059   
                            $\rho=0.8$  92  0.048  0.002  0.048   91   0.049   
         $n=2000$ $\pi=0.2$ $\rho=0.3$  95  0.120  0.013  0.120   94   0.123   
                            $\rho=0.8$  92  0.108  0.002  0.108   90   0.123   
                  $\pi=0.5$ $\rho=0.3$  95  0.038  0.003  0.038   95   0.039   
                            $\rho=0.8$  93  0.030  0.002  0.030   92   0.030   

                                                      ipw                \
                                         bias     std cov   rmse   bias   
1 abs    $n=500$  $\pi=0.2$ $\rho=0.3$  0.019   0.173  NA  0.146  0.017   
                            $\rho=0.8$  1.479  46.287  NA  0.132  0.016   
                  $\pi=0.5$ $\rho=0.3$  0.006   0.069  NA  0.062  0.004   
                            $\rho=0.8$  0.010   0.071  NA  0.061  0.007   
         $n=1000$ $\pi=0.2$ $\rho=0.3$  0.010   0.124  NA  0.115  0.009   
                            $\rho=0.8$  0.036   0.199  NA  0.125  0.011   
                  $\pi=0.5$ $\rho=0.3$  0.004   0.049  NA  0.048  0.003   
                            $\rho=0.8$  0.006   0.049  NA  0.048  0.003   
         $n=2000$ $\pi=0.2$ $\rho=0.3$  0.007   0.082  NA  0.081  0.005   
                            $\rho=0.8$  0.017   0.115  NA  0.094  0.009   
                  $\pi=0.5$ $\rho=0.3$  0.003   0.031  NA  0.034  0.002   
                            $\rho=0.8$  0.002   0.031  NA  0.034  0.001   
  2dpoly $n=500$  $\pi=0.2$ $\rho=0.3$  0.025   0.262  NA  0.267  0.100   
                            $\rho=0.8$  0.240  10.314  NA  0.213  0.138   
                  $\pi=0.5$ $\rho=0.3$  0.004   0.083  NA  0.106  0.049   
                            $\rho=0.8$  0.006   0.081  NA  0.092  0.035   
         $n=1000$ $\pi=0.2$ $\rho=0.3$  0.011   0.189  NA  0.211  0.054   
                            $\rho=0.8$  0.019   0.194  NA  0.169  0.083   
                  $\pi=0.5$ $\rho=0.3$  0.004   0.058  NA  0.073  0.016   
                            $\rho=0.8$  0.002   0.049  NA  0.068  0.018   
         $n=2000$ $\pi=0.2$ $\rho=0.3$  0.011   0.122  NA  0.154  0.022   
                            $\rho=0.8$  0.011   0.123  NA  0.124  0.047   
                  $\pi=0.5$ $\rho=0.3$  0.002   0.038  NA  0.056  0.006   
                            $\rho=0.8$  0.001   0.030  NA  0.051  0.009   

        

\begin{tabular}{lll||lll|lll|lll|lll|}
\toprule
 &  &  &  &  & \multicolumn{4}{c|}{dr} & \multicolumn{4}{c|}{tmle} & \multicolumn{4}{c|}{ipw} & \multicolumn{4}{c|}{direct} \\
 &  &  &  &  & cov & rmse & bias & std & cov & rmse & bias & std & cov & rmse & bias & std & cov & rmse & bias & std \\
\midrule
\multirow[t]{24}{*}{\textbf{1}} & \multirow[t]{12}{*}{\textbf{abs}} & \multirow[t]{4}{*}{\textbf{$n=500$}} & \multirow[t]{2}{*}{\textbf{$\pi=0.2$}} & \textbf{$\rho=0.3$} & 94 & 0.151 & 0.018 & 0.150 & 90 & 0.175 & 0.019 & 0.173 & NA & 0.146 & 0.017 & 0.145 & NA & 0.131 & 0.005 & 0.130 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{} & \textbf{$\rho=0.8$} & 90 & 0.147 & 0.010 & 0.146 & 80 & 46.310 & 1.479 & 46.287 & NA & 0.132 & 0.016 & 0.131 & NA & 0.109 & 0.020 & 0.108 \\
\cline{4-21}
\textbf{} & \textbf{} & \textbf{} & \multirow[t]{2}{*}{\textbf{$\pi=0.5$}} & \textbf{$\rho=0.3$} & 93 & 0.066 & 0.007 & 0.066 & 92 & 0.069 & 0.006 & 0.069 & NA & 0.062 & 0.004 & 0.062 & NA & 0.078 & 0.000 

FileNotFoundError: [Errno 2] No such file or directory: 'res_fn_abs_n_500_n_t_1_stregth_0.2_0.3_eps_0.1_clever_True_l2h_0.0004.jbl'

In [13]:
for clever in [False]:
    resd = {}
    for n_t in [1]:
        resd[n_t] = {}
        for fname in ['2dpoly']:
            resd[n_t][fname] = OrderedDict()
            for n in [2000, 20000]:
                lambda_l2_h = .1/n**(.9)
                nkey = f'$n={n}$'
                resd[n_t][fname][nkey] = {}
                for iv_strength in [0.05, 0.1]:
                    ivkey = f'$\pi={iv_strength}$'
                    resd[n_t][fname][nkey][ivkey] = {}
                    for endogeneity_strength in [0.05, 0.1]:
                        true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                    
                        ekey = f'$\rho={endogeneity_strength}$'
                        resd[n_t][fname][nkey][ivkey][ekey] = {}
                        for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                            data = np.array([r[it] for r in results])
                            if method in ['dr', 'tmle']:
                                cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                            else:
                                cov = 'NA'
                            resd[n_t][fname][nkey][ivkey][ekey][method] = {
                                            'cov': cov,
                                            'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                            'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}',
                                            'std': f'{np.std(data[:, 0]):.3f}'}
                        resd[n_t][fname][nkey][ivkey][ekey] = pd.concat({f'${true:.2f}$': pd.DataFrame(resd[n_t][fname][nkey][ivkey][ekey])})
                    resd[n_t][fname][nkey][ivkey] = pd.concat(resd[n_t][fname][nkey][ivkey], sort=False)
                resd[n_t][fname][nkey] = pd.concat(resd[n_t][fname][nkey], sort=False)
            resd[n_t][fname] = pd.concat(resd[n_t][fname], sort=False)
        resd[n_t] = pd.concat(resd[n_t], sort=False)
    display(pd.concat(resd).unstack(level=6))
    print(pd.concat(resd).unstack(level=6).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='lll||lll|lll|lll|lll|',
                                                    multicolumn_format='c|'))

FileNotFoundError: [Errno 2] No such file or directory: 'res_fn_2dpoly_n_2000_n_t_1_stregth_0.05_0.3_eps_0.1_clever_False_l2h_0.0001.jbl'

In [8]:
endogeneity_strength = 0.3
fname = 'cct'
for clever in [False]:
    resd = {}
    for n_t in [0, 5, 10]:
        resd[f'$n_x={n_t}$'] = {}
        for n in [1000, 5000]:
            lambda_l2_h = .1/n**(.9)
            nkey = f'$n={n}$'
            resd[f'$n_x={n_t}$'][nkey] = {}
            for iv_strength in [0.0, 0.5]:
                if n_t == 0 and iv_strength == 0.5:
                    continue

                true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                ivkey = "$\\tilde{\\rho}=" + f"{iv_strength}$"
                resd[f'$n_x={n_t}$'][nkey][ivkey] = {}
                for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                    if method == 'ipw':
                        continue
                    data = np.array([r[it] for r in results])
                    if method in ['dr', 'tmle']:
                        cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                    else:
                        cov = 'NA'
                    resd[f'$n_x={n_t}$'][nkey][ivkey][method] = {
                                    'cov': cov,
                                    'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                    'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}',
                                    'std': f'{np.std(data[:, 0]):.3f}'}
                resd[f'$n_x={n_t}$'][nkey][ivkey] = pd.DataFrame(resd[f'$n_x={n_t}$'][nkey][ivkey])
            resd[f'$n_x={n_t}$'][nkey] = pd.concat(resd[f'$n_x={n_t}$'][nkey], sort=False)
        resd[f'$n_x={n_t}$'] = pd.concat(resd[f'$n_x={n_t}$'], sort=False)
    display(pd.concat(resd).unstack(level=3))
    print(pd.concat(resd).unstack(level=3).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='lll||llll|llll|llll|',
                                                    multicolumn_format='c|'))

dr                      tmle         \
                                     cov   rmse   bias    std  cov   rmse   
$n_x=0$  $n=1000$ $\tilde{\rho}=0.0$  92  0.084  0.027  0.080   92  0.077   
         $n=5000$ $\tilde{\rho}=0.0$  94  0.032  0.008  0.031   94  0.034   
$n_x=5$  $n=1000$ $\tilde{\rho}=0.0$  86  0.181  0.050  0.174   82  0.202   
                  $\tilde{\rho}=0.5$  57  0.324  0.213  0.245   50  0.405   
         $n=5000$ $\tilde{\rho}=0.0$  55  0.143  0.092  0.109   53  0.161   
                  $\tilde{\rho}=0.5$  42  0.142  0.097  0.103   44  0.146   
$n_x=10$ $n=1000$ $\tilde{\rho}=0.0$  86  0.139  0.011  0.138   80  0.164   
                  $\tilde{\rho}=0.5$  62  0.241  0.116  0.211   56  0.317   
         $n=5000$ $\tilde{\rho}=0.0$  90  0.027  0.008  0.026   80  0.044   
                  $\tilde{\rho}=0.5$  48  0.120  0.085  0.085   57  0.117   

                                                   direct                       
                                       bias    std    cov   rmse   bias    std  
$n_x=0$  $n=1000$ $\tilde{\rho}=0.0$  0.029  0.072     NA  0.235  0.081  0.221  
         $n=5000$ $\tilde{\rho}=0.0$  0.010  0.032     NA  0.027  0.013  0.024  
$n_x=5$  $n=1000$ $\tilde{\rho}=0.0$  0.039  0.198     NA  0.313  0.131  0.285  
                  $\tilde{\rho}=0.5$  0.262  0.309     NA  0.331  0.020  0.330  
         $n=5000$ $\tilde{\rho}=0.0$  0.105  0.121     NA  0.065  0.006  0.064  
                  $\tilde{\rho}=0.5$  0.084  0.119     NA  0.173  0.160  0.065  
$n_x=10$ $n=1000$ $\tilde{\rho}=0.0$  0.025  0.162     NA  0.274  0.010  0.274  
                  $\tilde{\rho}=0.5$  0.155  0.277     NA  0.352  0.027  0.351  
         $n=5000$ $\tilde{\rho}=0.0$  0.026  0.036     NA  0.074  0.071  0.022  
                  $\tilde{\rho}=0.5$  0.058  0.101     NA  0.180  0.157  0.087

\begin{tabular}{lll||llll|llll|llll|}
\toprule
 &  &  & \multicolumn{4}{c|}{dr} & \multicolumn{4}{c|}{tmle} & \multicolumn{4}{c|}{direct} \\
 &  &  & cov & rmse & bias & std & cov & rmse & bias & std & cov & rmse & bias & std \\
\midrule
\multirow[t]{2}{*}{\textbf{$n_x=0$}} & \textbf{$n=1000$} & \textbf{$\tilde{\rho}=0.0$} & 92 & 0.084 & 0.027 & 0.080 & 92 & 0.077 & 0.029 & 0.072 & NA & 0.235 & 0.081 & 0.221 \\
\cline{2-15}
\textbf{} & \textbf{$n=5000$} & \textbf{$\tilde{\rho}=0.0$} & 94 & 0.032 & 0.008 & 0.031 & 94 & 0.034 & 0.010 & 0.032 & NA & 0.027 & 0.013 & 0.024 \\
\cline{1-15} \cline{2-15}
\multirow[t]{4}{*}{\textbf{$n_x=5$}} & \multirow[t]{2}{*}{\textbf{$n=1000$}} & \textbf{$\tilde{\rho}=0.0$} & 86 & 0.181 & 0.050 & 0.174 & 82 & 0.202 & 0.039 & 0.198 & NA & 0.313 & 0.131 & 0.285 \\
\textbf{} & \textbf{} & \textbf{$\tilde{\rho}=0.5$} & 57 & 0.324 & 0.213 & 0.245 & 50 & 0.405 & 0.262 & 0.309 & NA & 0.331 & 0.020 & 0.330 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n=50

In [9]:
endogeneity_strength = 0.3
fname = 'cct'
for clever in [False]:
    resd = {}
    for n_t in [0, 5, 10]:
        resd[f'$n_x={n_t}$'] = {}
        for n in [1000, 5000]:
            lambda_l2_h = .1/n**(.9)
            nkey = f'$n={n}$'
            resd[f'$n_x={n_t}$'][nkey] = {}
            for iv_strength in [0.0, 0.5]:
                if n_t == 0 and iv_strength == 0.5:
                    continue

                true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                ivkey = "$\\tilde{\\rho}=" + f"{iv_strength}$"
                resd[f'$n_x={n_t}$'][nkey][ivkey] = {}
                for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                    if method == 'ipw':
                        continue
                    data = np.array([r[it] for r in results])
                    confidence = .95
                    se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
                    confidence = .99
                    data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
                    data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
                    if method in ['dr', 'tmle']:
                        cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                    else:
                        cov = 'NA'
                    resd[f'$n_x={n_t}$'][nkey][ivkey][method] = {
                                    'cov': cov,
                                    'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                    'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}',
                                    'std': f'{np.std(data[:, 0]):.3f}'}
                resd[f'$n_x={n_t}$'][nkey][ivkey] = pd.DataFrame(resd[f'$n_x={n_t}$'][nkey][ivkey])
            resd[f'$n_x={n_t}$'][nkey] = pd.concat(resd[f'$n_x={n_t}$'][nkey], sort=False)
        resd[f'$n_x={n_t}$'] = pd.concat(resd[f'$n_x={n_t}$'], sort=False)
    display(pd.concat(resd).unstack(level=3))
    print(pd.concat(resd).unstack(level=3).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='lll||llll|llll|llll|',
                                                    multicolumn_format='c|'))

dr                      tmle         \
                                     cov   rmse   bias    std  cov   rmse   
$n_x=0$  $n=1000$ $\tilde{\rho}=0.0$  98  0.084  0.027  0.080   98  0.077   
         $n=5000$ $\tilde{\rho}=0.0$  99  0.032  0.008  0.031   99  0.034   
$n_x=5$  $n=1000$ $\tilde{\rho}=0.0$  95  0.181  0.050  0.174   92  0.202   
                  $\tilde{\rho}=0.5$  73  0.324  0.213  0.245   64  0.405   
         $n=5000$ $\tilde{\rho}=0.0$  68  0.143  0.092  0.109   66  0.161   
                  $\tilde{\rho}=0.5$  54  0.142  0.097  0.103   56  0.146   
$n_x=10$ $n=1000$ $\tilde{\rho}=0.0$  94  0.139  0.011  0.138   90  0.164   
                  $\tilde{\rho}=0.5$  74  0.241  0.116  0.211   68  0.317   
         $n=5000$ $\tilde{\rho}=0.0$  97  0.027  0.008  0.026   90  0.044   
                  $\tilde{\rho}=0.5$  61  0.120  0.085  0.085   68  0.117   

                                                   direct                       
                                       bias    std    cov   rmse   bias    std  
$n_x=0$  $n=1000$ $\tilde{\rho}=0.0$  0.029  0.072     NA  0.235  0.081  0.221  
         $n=5000$ $\tilde{\rho}=0.0$  0.010  0.032     NA  0.027  0.013  0.024  
$n_x=5$  $n=1000$ $\tilde{\rho}=0.0$  0.039  0.198     NA  0.313  0.131  0.285  
                  $\tilde{\rho}=0.5$  0.262  0.309     NA  0.331  0.020  0.330  
         $n=5000$ $\tilde{\rho}=0.0$  0.105  0.121     NA  0.065  0.006  0.064  
                  $\tilde{\rho}=0.5$  0.084  0.119     NA  0.173  0.160  0.065  
$n_x=10$ $n=1000$ $\tilde{\rho}=0.0$  0.025  0.162     NA  0.274  0.010  0.274  
                  $\tilde{\rho}=0.5$  0.155  0.277     NA  0.352  0.027  0.351  
         $n=5000$ $\tilde{\rho}=0.0$  0.026  0.036     NA  0.074  0.071  0.022  
                  $\tilde{\rho}=0.5$  0.058  0.101     NA  0.180  0.157  0.087

\begin{tabular}{lll||llll|llll|llll|}
\toprule
 &  &  & \multicolumn{4}{c|}{dr} & \multicolumn{4}{c|}{tmle} & \multicolumn{4}{c|}{direct} \\
 &  &  & cov & rmse & bias & std & cov & rmse & bias & std & cov & rmse & bias & std \\
\midrule
\multirow[t]{2}{*}{\textbf{$n_x=0$}} & \textbf{$n=1000$} & \textbf{$\tilde{\rho}=0.0$} & 98 & 0.084 & 0.027 & 0.080 & 98 & 0.077 & 0.029 & 0.072 & NA & 0.235 & 0.081 & 0.221 \\
\cline{2-15}
\textbf{} & \textbf{$n=5000$} & \textbf{$\tilde{\rho}=0.0$} & 99 & 0.032 & 0.008 & 0.031 & 99 & 0.034 & 0.010 & 0.032 & NA & 0.027 & 0.013 & 0.024 \\
\cline{1-15} \cline{2-15}
\multirow[t]{4}{*}{\textbf{$n_x=5$}} & \multirow[t]{2}{*}{\textbf{$n=1000$}} & \textbf{$\tilde{\rho}=0.0$} & 95 & 0.181 & 0.050 & 0.174 & 92 & 0.202 & 0.039 & 0.198 & NA & 0.313 & 0.131 & 0.285 \\
\textbf{} & \textbf{} & \textbf{$\tilde{\rho}=0.5$} & 73 & 0.324 & 0.213 & 0.245 & 64 & 0.405 & 0.262 & 0.309 & NA & 0.331 & 0.020 & 0.330 \\
\cline{2-15}
\textbf{} & \multirow[t]{2}{*}{\textbf{$n=50